# Relatório de Resultados dos Pagamentos

# Importando as bibliotecas, definindo as funções e carregando as tabelas necessárias

In [1]:
#Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import datetime
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os

In [2]:
#Cria as pastas necessárias
pasta ="./relatorio/pags"

access_rights = 0o777

try:
    os.mkdir('relatorio', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (relatorio) falhou ou a pasta já existe")
else:
    print ("\tPasta (relatorio) criada com sucesso!")

try:
    os.mkdir('relatorio/pags', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (pags) falhou ou a pasta já existe")
else:
    print ("\tPasta (pags) criada com sucesso!")

	AVISO: A criação da pasta (relatorio) falhou ou a pasta já existe
	AVISO: A criação da pasta (pags) falhou ou a pasta já existe


In [3]:
#Define as funções personalizadas necessárias
def write_to_html_file(df, title='', filename='out.html'):
    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    body{ 
        display: flex;
        align-items: center;
        justify-content: center;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
        width: 90%;
        color: #707070;
    }
    table, th, td {
        border: 1px solid #707070;
        border-collapse: collapse;
    }
    th {
        padding: 5px;
        text-align: left;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    td {
        padding: 5px;
        text-align: right;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    if type(df) == pd.io.formats.style.Styler:
        result += df.render()
    else:
        result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

formatacao = {'Totais dos Pagamentos':"R$ {:,.2f}", 'Pagamentos Atrasados':"R$ {:,.2f}", 'Pagamentos em Dia':"R$ {:,.2f}",
              'Pagamentos Adiantados':"R$ {:,.2f}", 'Número de Pagamentos':"{:,.0f}", 'Número de Clientes':"{:,.0f}",
             'Número de Clientes Atrasados':"{:,.0f}",'Número de Clientes em Dia':"{:,.0f}", 'Número de Clientes Adiantados':"{:,.0f}",
             'Número de Pagamentos Atrasados':"{:,.0f}",'Número de Pagamentos em Dia':"{:,.0f}", 'Número de Pagamentos Adiantados':"{:,.0f}",
             'Número de Pagamentos por Cliente':"{:,.2f}"}

formata_pct = {'% Atrasados':"{:.2f}%", '% em Dia':"{:.2f}%", '% Adiantados':"{:.2f}%"}

def highlight_max(s):    
    is_max = s == s.max()
    return ['color: white; background-color: #3749E9' if v else '' for v in is_max]

def highlight_min(s):
    is_max = s == s.min()
    return ['color: white; background-color: #112244' if v else '' for v in is_max]

def destaque(val):
    color = '#F2F200' if val == 'Sim' else ''
    return 'background: {}'.format(color)

def destaque_coluna(val):
    color = '#F2F200' if val == 0 else ''
    return 'background: {}'.format(color)

In [4]:
#Carregas os dataframes
df = pd.read_excel ('STG_PGT.xlsx')
cf = pd.read_excel ('STG_MDL.xlsx')
bf = pd.read_excel ('STG_FNT_ITT.xlsx')

In [5]:
#df

In [6]:
#df.dtypes

In [7]:
#Trata as datas
df.DAT_VCT = pd.to_datetime(df.DAT_VCT, format='%d%m%Y', errors='coerce')
df.DAT_RSS_FNT_ITT = pd.to_datetime(df.DAT_RSS_FNT_ITT, format='%Y-%m-%d', errors='coerce')
df.DAT_INC_DBO = pd.to_datetime(df.DAT_INC_DBO, format='%Y-%m-%d', errors='coerce')

#Completa espaços vazios com zero
df = df.loc[:, df.columns != ('DAT_VCT', 'DAT_RSS_FNT_ITT', 'DAT_INC_DBO')]
df.fillna(0)

#Converte as colunas para os valores corretos
df.ID_STG_PGT = df.ID_STG_PGT.astype (np.int64)
df.VLR_PGT_FAT = df.VLR_PGT_FAT.astype (np.int64)
df.COD_MDL = df.COD_MDL.astype(str)
df.QTD_CLI_CAD_POS = df.QTD_CLI_CAD_POS.astype(np.int64)
df.QTD_PGT = df.QTD_PGT.astype(np.int64)
df.ID_FNT_ITT = df.ID_FNT_ITT.astype (np.int64)
df.DES_TIP_PSS = df.DES_TIP_PSS.astype (str)
#df.dtypes

# Analisando o valor dos pagamentos vencidos por modalidades

In [8]:
#Converte todos os valores para acrescentar dois decimais
for c in df.VLR_PGT_FAT:
    d = c * 0.01
    df.VLR_PGT_FAT = df.VLR_PGT_FAT.replace(c, d)

In [9]:
#Escolhe as colunas necessárias.
new = cf[['COD_MDL', 'DES_MDL']]

#Converte em um dicionário (valores em lista).
dictionary = new.set_index('COD_MDL').T.to_dict('list')

#Converte os valores (em lista) para string.
for key, value in dictionary.items():
    dictionary[key] = str(value).strip("['']")

#Verifica se existe no dicionário e adiciona a uma lista
descricao = []
for resp in df['COD_MDL']:
    if resp in dictionary:
        descricao.append(dictionary[resp])
    else:
        descricao.append(f'{resp} (MODALIDADE NAO ENCONTRADA)')

#Cria uma nova coluna com as descrições encontradas
df['DES_MDL'] = descricao

In [10]:
#Compara a data de vencimento com a data de alteração e exclui as linhas com datas não vencidas
vencidos = df
ts = pd.Timestamp
for x in range (len (df.DAT_VCT)):
    if ts.date(df.DAT_VCT[x]) > ts.date(df.DAT_RSS_FNT_ITT[x]) or (str(df.DAT_VCT[x]) == 'NaT'):
        vencidos = vencidos.drop (x)

#print (len (vencidos))
novo_vencidos = vencidos

#Cria um dataframe
vencidos = vencidos[['DES_MDL', 'VLR_PGT_FAT', 'QTD_CLI_CAD_POS', 'QTD_PGT']]
vencidos = vencidos.groupby('DES_MDL').sum()
vencidos = vencidos.rename(columns={'VLR_PGT_FAT': 'Totais dos Pagamentos', 'QTD_PGT': 'Número de Pagamentos',
                                'QTD_CLI_CAD_POS': 'Número de Clientes'}).sort_values(by = 'Totais dos Pagamentos', ascending = True)
vencidos = vencidos.rename_axis('Modalidades', axis='columns')
vencidos.index.name = None

pgt_por_cli = [(vencidos['Número de Pagamentos'][x] / vencidos['Número de Clientes'][x]) for x in range (len (vencidos))]
vencidos['Número de Pagamentos por Cliente'] = pgt_por_cli

visualiza = vencidos.replace(to_replace = 0, value = np.nan)

visualiza = (visualiza
                .style
                .format(formatacao, na_rep="-")
                .highlight_null(null_color='#F2F200')
                .apply(highlight_max)
                .apply(highlight_min))
write_to_html_file(visualiza, title='', filename=pasta + "/pgt_tab_vencidos.html")
visualiza

In [11]:
#Define escalas para o tamanho máximo padrão dos gráficos
vencidos_range = [0, round(max(vencidos['Totais dos Pagamentos']))]

#Filtra valores menores ou iguais a 100 milhões
vencidos_menor = vencidos[vencidos['Totais dos Pagamentos'] > 0]
vencidos_menor = vencidos_menor[vencidos_menor['Totais dos Pagamentos'] <= 100000000]
vencidos_menor = vencidos_menor.sort_values(by = 'Totais dos Pagamentos', ascending = True)

#Filtra valores maiores que 100 milhões
vencidos_maior = vencidos[vencidos['Totais dos Pagamentos'] > 100000000]
vencidos_maior = vencidos_maior.sort_values(by = 'Totais dos Pagamentos', ascending = False)

#Plota em formato gráfico e exporta para HTML
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.7])

fig.add_trace(go.Bar(x=vencidos_menor.index,
           y=vencidos_menor['Totais dos Pagamentos'].dropna(),
           marker=dict(color='#3749E9')
          ),
    row=1, col=1
)

fig.add_trace(go.Bar(x = vencidos_maior.index,
           y = vencidos_maior['Totais dos Pagamentos'].dropna(),
           marker=dict(color='#3749E9')
          ),
    row=1, col=2
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_yaxes(range=vencidos_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)
fig.write_html(pasta + "/pgt_pagamentos_vencidos.html")

# Verificando o valor total de pagamentos por modalidades

In [12]:
#Cria um dataframe
total_pagamentos = df[['DES_MDL', 'VLR_PGT_FAT']]
total_pagamentos = total_pagamentos.groupby('DES_MDL').sum().sort_values(by = 'VLR_PGT_FAT')
#total_pagamentos

In [13]:
#Define escalas para o tamanho máximo padrão dos gráficos
total_pagamentos_range = [0, round(max(total_pagamentos.VLR_PGT_FAT))]

#Filtra valores menores ou iguais a 100 milhões
total_pagamentos_menor = total_pagamentos[total_pagamentos['VLR_PGT_FAT'] > 0]
total_pagamentos_menor = total_pagamentos_menor[total_pagamentos_menor['VLR_PGT_FAT'] <= 100000000]
total_pagamentos_menor = total_pagamentos_menor.sort_values(by = 'VLR_PGT_FAT', ascending = True)

#Filtra valores maiores que 100 milhões
total_pagamentos_maior = total_pagamentos[total_pagamentos['VLR_PGT_FAT'] > 100000000]
total_pagamentos_maior = total_pagamentos_maior.sort_values(by = 'VLR_PGT_FAT', ascending = False)

#Plota em formato gráfico e exporta para HTML
fig = make_subplots(rows=1, cols=2, column_widths=[0.6, 0.6])

fig.add_trace(go.Bar(x=total_pagamentos_menor.index,
           y=total_pagamentos_menor['VLR_PGT_FAT'].dropna(),
           marker=dict(color='#112244')
          ),
    row=1, col=1
)

fig.add_trace(go.Bar(x = total_pagamentos_maior.index,
           y = total_pagamentos_maior['VLR_PGT_FAT'].dropna(),
           marker=dict(color='#112244')
          ),
    row=1, col=2
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_yaxes(range=total_pagamentos_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)
fig.write_html(pasta + "/pgt_total_pagamentos.html")

# Comparando os Valores de Pagamentos Vencidos e Totais

In [14]:
#Plota em formato gráfico e exporta para HTML
data = [go.Bar(x=vencidos.index,
               y=vencidos['Totais dos Pagamentos'],
               name='Pagamentos Vencidos',
               marker=dict(color='#3749E9')
              ),
        go.Bar(x=total_pagamentos.index,
               y=total_pagamentos['VLR_PGT_FAT'],
               name='Pagamentos Totais',
               marker=dict(color='#112244')
              )]

fig = go.Figure(data=data)

fig = go.Figure(data=data)
fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(xaxis={'categoryorder':'total ascending'}, showlegend=True)

iplot(fig)
fig.write_html(pasta + "/pgt_vencidos_totais.html")

# Analisando o número total de pagamentos por modalidades

In [15]:
#Cria um dataframe
num_pagamentos = df[['DES_MDL', 'QTD_PGT']]
num_pagamentos = num_pagamentos.groupby('DES_MDL').sum().sort_values(by = 'QTD_PGT')
#num_pagamentos

In [16]:
#Define escalas para o tamanho máximo padrão dos gráficos
num_pagamentos_range = [0, round(max(num_pagamentos.QTD_PGT))]

#Filtra valores menores ou iguais a 10 mil
num_pagamentos_menor = num_pagamentos[num_pagamentos['QTD_PGT'] > 0]
num_pagamentos_menor = num_pagamentos_menor[num_pagamentos_menor['QTD_PGT'] <= 10000]
num_pagamentos_menor = num_pagamentos_menor.sort_values(by = 'QTD_PGT', ascending = True)

#Filtra valores maiores que 10 mil e menores ou iguais a 1 milhão
num_pagamentos_medio = num_pagamentos[num_pagamentos['QTD_PGT'] > 10000]
num_pagamentos_medio = num_pagamentos_medio[num_pagamentos_medio['QTD_PGT'] <= 1000000]
num_pagamentos_medio = num_pagamentos_medio.sort_values(by = 'QTD_PGT', ascending = True)

#Filtra valores maiores que 1 milhão
num_pagamentos_maior = num_pagamentos[num_pagamentos['QTD_PGT'] > 1000000]
num_pagamentos_maior = num_pagamentos_maior.sort_values(by = 'QTD_PGT', ascending = True)

#Plota em formato gráfico e exporta para HTML
fig = make_subplots(rows=1, cols=3, column_widths=[0.4, 0.5, 0.5])

fig.add_trace(go.Bar(x=num_pagamentos_menor.index,
           y=num_pagamentos_menor['QTD_PGT'].dropna(),
           marker=dict(color='#112244')
          ),
    row=1, col=1
)

fig.add_trace(go.Bar(x = num_pagamentos_medio.index,
           y = num_pagamentos_medio['QTD_PGT'].dropna(),
           marker=dict(color='#112244')
          ),
    row=1, col=2
)

fig.add_trace(go.Bar(x = num_pagamentos_maior.index,
           y = num_pagamentos_maior['QTD_PGT'].dropna(),
           marker=dict(color='#112244')
          ),
    row=1, col=3
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_yaxes(range=num_pagamentos_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)
fig.write_html(pasta + "/pgt_num_pagamentos.html")

# Contando o número de clientes por modalidades

In [17]:
#Cria um dataframe
clientes = df[['DES_MDL', 'QTD_CLI_CAD_POS']]
clientes = clientes.groupby('DES_MDL').sum().sort_values(by = 'QTD_CLI_CAD_POS')
#clientes

In [18]:
#Define escalas para o tamanho máximo padrão dos gráficos
clientes_range = [0, round(max(clientes.QTD_CLI_CAD_POS))]

#Filtra valores menores ou iguais a 10 mil
clientes_menor = clientes[clientes['QTD_CLI_CAD_POS'] > 0]
clientes_menor = clientes_menor[clientes_menor['QTD_CLI_CAD_POS'] <= 10000]
clientes_menor = clientes_menor.sort_values(by = 'QTD_CLI_CAD_POS', ascending = True)

#Filtra valores maiores que 10 mil e menores ou iguais a 1 milhão
clientes_medio = clientes[clientes['QTD_CLI_CAD_POS'] > 10000]
clientes_medio = clientes_medio[clientes_medio['QTD_CLI_CAD_POS'] <= 1000000]
clientes_medio = clientes_medio.sort_values(by = 'QTD_CLI_CAD_POS', ascending = True)

#Filtra valores maiores que 1 milhão
clientes_maior = clientes[clientes['QTD_CLI_CAD_POS'] > 1000000]
clientes_maior = clientes_maior.sort_values(by = 'QTD_CLI_CAD_POS', ascending = True)

#Plota em formato gráfico e exporta para HTML
fig = make_subplots(rows=1, cols=3, column_widths=[0.4, 0.5, 0.5])

fig.add_trace(go.Bar(x=clientes_menor.index,
           y=clientes_menor['QTD_CLI_CAD_POS'].dropna(),
           marker=dict(color='#00B7CC')
          ),
    row=1, col=1
)

fig.add_trace(go.Bar(x = clientes_medio.index,
           y = clientes_medio['QTD_CLI_CAD_POS'].dropna(),
           marker=dict(color='#00B7CC')
          ),
    row=1, col=2
)

fig.add_trace(go.Bar(x = clientes_maior.index,
           y = clientes_maior['QTD_CLI_CAD_POS'].dropna(),
           marker=dict(color='#00B7CC')
          ),
    row=1, col=3
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_yaxes(range=clientes_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)
fig.write_html(pasta + "/pgt_clientes.html")

# Comparando o número de clientes com o número de pagamentos

In [19]:
#Plota em formato gráfico e exporta para HTML
data = [go.Bar(x=clientes.index,
               y=clientes.QTD_CLI_CAD_POS,
               name='Número de Clientes',
               marker=dict(color='#00B7CC')
              ),
        go.Bar(x=num_pagamentos.index,
               y=num_pagamentos.QTD_PGT,
               name='Número de Pagamentos',
               marker=dict(color='#112244')
              )]

fig = go.Figure(data=data)

fig = go.Figure(data=data)
fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(xaxis={'categoryorder':'total ascending'}, showlegend=True)

iplot(fig)
fig.write_html(pasta + "/pgt_clientes_num_pagamentos.html")

# Gerando uma tabela geral dos pagamentos por modalidades

In [20]:
#Cria um dataframe geral com as colunas de totais de pagamentos, clientes, e número de pagamentos
tabela = df[['DES_MDL', 'VLR_PGT_FAT', 'QTD_CLI_CAD_POS', 'QTD_PGT']]
tabela = tabela.groupby('DES_MDL').sum()
tabela = tabela.rename(columns={'VLR_PGT_FAT': 'Totais dos Pagamentos', 'QTD_PGT': 'Número de Pagamentos',
                                'QTD_CLI_CAD_POS': 'Número de Clientes'}).sort_values(by = 'Totais dos Pagamentos', ascending = True)
tabela = tabela.rename_axis('Modalidades', axis='columns')
tabela.index.name = None

pgt_por_cli = [(tabela['Número de Pagamentos'][x] / tabela['Número de Clientes'][x]) for x in range (len (tabela))]
tabela['Número de Pagamentos por Cliente'] = pgt_por_cli

visualiza = tabela.replace(to_replace = 0, value = np.nan)

visualiza = (visualiza
                .style
                .format(formatacao, na_rep="-")
                .highlight_null(null_color='#F2F200')
                .apply(highlight_max)
                .apply(highlight_min))
write_to_html_file(visualiza, title='', filename=pasta + "/pgt_tab_modalidades.html")
visualiza

# Calculando os indicadores gerais

In [21]:
#Calcula os resultados totais da remessa
remessa_total_pagamentos = df.VLR_PGT_FAT.sum()
remessa_num_pagamentos = df.QTD_PGT.sum()
remessa_num_vencidos = novo_vencidos.QTD_PGT.sum()
remessa_pgt_por_cli = tabela['Número de Pagamentos'].sum() / tabela['Número de Clientes'].sum()

#remessa_total_pagamentos, remessa_num_pagamentos, remessa_num_vencidos, remessa_pgt_por_cli